In [1]:
import scanpy as sc
import scanpy.external as sce
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
import mygene
from scipy.stats import median_abs_deviation

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message="ChainedAssignmentError")

In [3]:
base_dir = Path().resolve().parent
base_dir

PosixPath('/home/sadegh/python_projects/HRplus-BC-Multimodal')

### Load Data and Find Gene Symbols

In [4]:
# Load annotated data and model gene data
adata = sc.read_h5ad(base_dir / "dataset/h5ad/gene_expression_annotated.h5ad")
print(f"Initial Dimension: {adata.n_obs} cells, {adata.n_vars} genes")
gene_meta = pd.read_csv(base_dir / 'dataset/csv/model_genes.csv')

Initial Dimension: 58177 cells, 36601 genes


In [5]:
adata

AnnData object with n_obs × n_vars = 58177 × 36601
    obs: 'sample_id', 'patient_id', 'response'

In [6]:
adata_before = adata.copy()

In [7]:
# show a few var rows to see what you have
print(adata.var.head(10))
print("any var names starting with 'MT-':", adata.var_names.str.startswith("MT-").sum())
print("any var names starting with 'mt-':", adata.var_names.str.startswith("mt-").sum())
# if gene symbols are in a separate column, show that column (common names: 'gene_ids' 'gene_symbols' or 'features')
print(adata.var.columns)
for col in adata.var.columns:
    if adata.var[col].dtype == object:
        s = adata.var[col].astype(str).str.startswith("MT-").sum()
        if s>0:
            print(f"column {col} has {s} MT- entries")

Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945, ENSG00000239906, ENSG00000241860, ENSG00000241599, ENSG00000286448, ENSG00000236601]
any var names starting with 'MT-': 0
any var names starting with 'mt-': 0
Index([], dtype='object')


In [8]:
mg = mygene.MyGeneInfo()

# Query gene symbols
ids = adata.var_names.tolist()
query = mg.querymany(ids, scopes='ensembl.gene', fields='symbol', species='human', as_dataframe=True)

# Extract symbol mapping, reindex to adata.var_names
symbol_map = query['symbol'].to_dict()

# Assign mapped symbols safely
adata.var['symbol'] = adata.var_names.map(symbol_map)

# Check mapping coverage
mapped = adata.var['symbol'].notnull().sum()
total = adata.var.shape[0]
print(f"Mapped {mapped}/{total} Ensembl IDs to gene symbols ({mapped/total:.1%})")

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
28 input query terms found dup hits:	[('ENSG00000228044', 2), ('ENSG00000226506', 2), ('ENSG00000261600', 2), ('ENSG00000234162', 2), ('E
1148 input query terms found no hit:	['ENSG00000238009', 'ENSG00000230699', 'ENSG00000241180', 'ENSG00000236948', 'ENSG00000271895', 'ENS


Mapped 26859/36601 Ensembl IDs to gene symbols (73.4%)


### Apply QC on Transcriptomics Data

In [ ]:
adata.var['symbol'] = adata.var['symbol'].astype(str)

# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
adata.var['mt'] = adata.var['symbol'].str.upper().str.startswith('MT-')
print("Number of mitochondrial genes found:", adata.var['mt'].sum())

# ribosomal genes
adata.var["ribo"] = adata.var['symbol'].str.upper().str.startswith(("RPS", "RPL"))

# hemoglobin genes. Pls note this dataset isperipheral blood
adata.var["hb"] = adata.var['symbol'].str.upper().str.startswith("^HB[^(P)]")

In [ ]:
# mitochondrial genes are flagged
adata.var['mt'] = adata.var['symbol'].astype(str).str.upper().str.startswith('MT-')

# Compute QC metrics, including mitochondrial percentage
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True,  log1p=True, percent_top=[20])

print(adata.obs['pct_counts_mt'].describe())

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
# sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
p2 = sc.pl.violin(adata, "pct_counts_mt")
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

Filter low quality genes

In [ ]:
# Quality control
# Follow: https://scanpy-tutorials.readthedocs.io/en/latest/basic-scrna-tutorial.html
sc.pp.filter_cells(adata, min_genes=250)
sc.pp.filter_genes(adata, min_cells=3)

# Remove outliers if any
# https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier


adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
print(adata.obs.outlier.value_counts())
adata.obs["mt_outlier"] =  adata.obs["pct_counts_mt"] > 20

print(adata.obs.mt_outlier.value_counts())

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
p2 = sc.pl.violin(adata, "pct_counts_mt")
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

Checking for doublets and handling

In [ ]:
# Make a copy to store results
adata_scrub = adata.copy()

# Store doublet predictions
adata_scrub.obs['doublet_score'] = 0.0
adata_scrub.obs['predicted_doublet'] = False

# Process per batch to save memory
batches = adata_scrub.obs['sample_id'].unique()
for batch in batches:
    print(f"Processing batch: {batch}")
    adata_batch = adata_scrub[adata_scrub.obs['sample_id'] == batch].copy()

    # Run Scrublet
    sce.pp.scrublet(
        adata_batch,
        batch_key=None,  # batch already subsetted
        expected_doublet_rate=0.06,
        sim_doublet_ratio=1.0,
        n_prin_comps=20
    )

    # Assign back to main AnnData
    adata_scrub.obs.loc[adata_batch.obs_names, 'doublet_score'] = adata_batch.obs['doublet_score']
    adata_scrub.obs.loc[adata_batch.obs_names, 'predicted_doublet'] = adata_batch.obs['predicted_doublet']

In [ ]:
# Optional: remove predicted doublets
adata_filtered = adata_scrub[~adata_scrub.obs['predicted_doublet']].copy()

# Summary
n_doublets = adata_scrub.obs['predicted_doublet'].sum()
print(f"Total predicted doublets: {n_doublets}/{adata_scrub.n_obs} ({n_doublets/adata_scrub.n_obs:.2%})")
print(f"Cells after removing doublets: {adata_filtered.n_obs}")

In [ ]:
sns.histplot(adata_scrub.obs['doublet_score'], bins=100, color='skyblue')
plt.axvline(0.25, color='red', linestyle='--', label='threshold')
plt.xlabel('Doublet score')
plt.ylabel('Number of cells')
plt.legend()
plt.show()

In [ ]:
# Subset to remaining cells (after doublet removal)
adata_after = adata_scrub[~adata_scrub.obs['predicted_doublet']].copy()

# Plot histogram
plt.figure(figsize=(8,5))
sns.histplot(adata_after.obs['doublet_score'], bins=100, color='lightgreen')
plt.axvline(0.25, color='red', linestyle='--', label='typical threshold')
plt.xlabel('Doublet score')
plt.ylabel('Number of cells')
plt.title('Doublet score distribution after removing predicted doublets')
plt.legend()
plt.show()

In [ ]:
# Before removing doublets
total_cells_before = adata_scrub.n_obs
predicted_doublets_before = adata_scrub.obs['predicted_doublet'].sum()
singlets_before = total_cells_before - predicted_doublets_before
mean_score_before = adata_scrub.obs['doublet_score'].mean()
median_score_before = adata_scrub.obs['doublet_score'].median()

print("--- Before removing doublets ---")
print(f"Total cells: {total_cells_before}")
print(f"Predicted doublets: {predicted_doublets_before}")
print(f"Predicted singlets: {singlets_before}")
print(f"Mean doublet score: {mean_score_before:.4f}")
print(f"Median doublet score: {median_score_before:.4f}")
print()

# After removing doublets
adata_after = adata_scrub[~adata_scrub.obs['predicted_doublet']].copy()
total_cells_after = adata_after.n_obs
mean_score_after = adata_after.obs['doublet_score'].mean()
median_score_after = adata_after.obs['doublet_score'].median()

print("--- After removing doublets ---")
print(f"Total cells: {total_cells_after}")
print(f"Mean doublet score: {mean_score_after:.4f}")
print(f"Median doublet score: {median_score_after:.4f}")


In [ ]:
adata_after.layers["counts"] = adata_after.X.copy()
print(f"Raw counts saved. Layer keys: {adata_after.layers.keys()}")

In [ ]:
adata_after

In [ ]:
# adata_after.write_h5ad("gene_expression_processed1.h5ad")

### Generate Final Transcriptomics Data for 5000 HVG

In [9]:
adata = sc.read_h5ad(base_dir / "dataset/h5ad/gene_expression_processed1.h5ad")

In [10]:
adata

AnnData object with n_obs × n_vars = 52831 × 22735
    obs: 'sample_id', 'patient_id', 'response', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'outlier', 'mt_outlier', 'doublet_score', 'predicted_doublet'
    var: 'symbol', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    layers: 'counts'

In [11]:
# --- Normalization and log-transform ---
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# --- Compute HVGs but do NOT subset (retain all genes) ---
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=False)

# --- Print HVG summary stats (optional but good for verification) ---
print("Highly Variable Genes selection counts:")
print(adata.var['highly_variable'].value_counts())
print("\nSummary statistics of gene means and dispersions:")
print(adata.var[['means', 'dispersions', 'dispersions_norm']].describe())

# --- Plot HVG selection curve ---
# sc.pl.highly_variable_genes(adata, save=False)

Highly Variable Genes selection counts:
highly_variable
False    17735
True      5000
Name: count, dtype: int64

Summary statistics of gene means and dispersions:
              means   dispersions  dispersions_norm
count  2.273500e+04  22727.000000      22727.000000
mean   1.613837e-01      1.376082          0.000044
std    3.699218e-01      0.445417          0.999604
min    1.000000e-12     -0.615357         -4.533753
25%    1.435658e-03      1.220120         -0.404624
50%    2.512826e-02      1.337774         -0.074510
75%    1.764122e-01      1.473191          0.240613
max    5.249661e+00      6.398817         12.816979


In [12]:
hvg_check = set(adata.var[adata.var['highly_variable']==True].index) &set(gene_meta.Gene) 
print(f"Number of HVG genes: {len(hvg_check)}")

Number of HVG genes: 75


In [13]:
adata_hvg = adata[:, adata.var['highly_variable']].copy()
adata_hvg

AnnData object with n_obs × n_vars = 52831 × 5000
    obs: 'sample_id', 'patient_id', 'response', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'outlier', 'mt_outlier', 'doublet_score', 'predicted_doublet'
    var: 'symbol', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'

In [ ]:
# Convert to dense matrix if it's sparse
X = adata_hvg.X.toarray() if hasattr(adata_hvg.X, "toarray") else adata_hvg.X

# Create DataFrame (rows = cells, columns = genes)
df = pd.DataFrame(X, index=adata_hvg.obs_names, columns=adata_hvg.var_names)

# Save to CSV
df.to_csv(base_dir / "dataset/csv/transcriptomic_features.csv")

In [ ]:
# Sort genes by normalized dispersion (descending)
hvg_sorted = adata.var[adata.var['highly_variable']].sort_values(
    by='dispersions_norm', ascending=False
)

# Extract gene names in sorted order
hvg_genes = hvg_sorted.index.to_list()

# Save with column name 'gene'
pd.DataFrame({'gene': hvg_genes}).to_csv(base_dir / "dataset/csv/hvg_genes.csv", index=False)


### DE Analysis

In [ ]:
# DE genes
# DE analysis
# find DE genes by t-test
key_DE = "response"
sc.tl.rank_genes_groups(adata, key_DE, method="t-test", key_added=key_DE, reference='Non-responder') 

results = adata.uns[key_DE]
('0', '1', '2', '3', '4')

out = np.array([[0,0,0,0,0]]) 
for group in results['names'].dtype.names:
    out = np.vstack((out, np.vstack((results['names'][group],
                                     results['scores'][group],
                                     results['pvals_adj'][group],
                                     results['logfoldchanges'][group],
                                     np.array([group] * len(results['names'][group])).astype('object'))).T))



markers = pd.DataFrame(out[1:], columns = ['Gene', 'scores', 'pval_adj', 'lfc', 'cluster'])



In [ ]:
markers.query('Gene in @gene_meta.Gene & pval_adj < 0.05').to_csv('DE_metabolic_genes.csv')
markers.query(' pval_adj < 0.05').to_csv('DE_genes.csv')

In [ ]:
union_genes = set(markers.query('Gene in @gene_meta.Gene & pval_adj < 0.05').Gene) |hvg_check
pd.DataFrame(union_genes).to_csv('union_5000HVG_DE_genes.csv')

In [ ]:
print("Total metabolic genes in the model:", len(gene_meta))
print("Total metabolic genes if 5000HVG genes selected:", len(hvg_check))
print("Total DE metabolic genes :", len(set(markers.query('Gene in @gene_meta.Gene & pval_adj < 0.05').Gene)))
print("Total metabolic genes in union:", len(set(markers.query('Gene in @gene_meta.Gene & pval_adj < 0.05').Gene) |hvg_check))


In [ ]:
# Select only HVGs for downstream analysis
adata_hvg = adata[:, adata.var['highly_variable']].copy()

In [ ]:
adata_hvg